In [44]:
import json
import codecs
import pandas as pd
import networkx as nx

from gensim import corpora, models

from IPython.display import display, HTML

In [2]:
G = nx.read_gexf("datasets/influences.gexf")
communities = json.load(codecs.open("datasets/communities/louvain.json","r", "utf-8"))

In [ ]:
def get_sentences(doc_name):
  sentences = []
  # sentences = models.word2vec.LineSentence("datasets/sentences/%s.txt" % doc_name.replace("/", "-"))
  sentences = codecs.open("datasets/sentences/%s.txt" % doc_name.replace("/", "-"),"r", "utf-8").read().split("\n")  
  sentences = [ s.split() for s in sentences ]
  
  return sentences

def community_word2vec(community):
  sentences = reduce(lambda a,b: a + b, map(get_sentences, community))
  
  bigram = models.phrases.Phrases(sentences)
  
  model = models.word2vec.Word2Vec(bigram[sentences], batch_words=100)
  
  return model

for k in sorted([ int(k) for k in communities.keys()]):
#for k in ["16"]:
  print "processing community k=%s" % k

  m = community_word2vec(communities[str(k)])
  m.init_sims(replace=True)
  m.save("models/word2vec/community.%s.txt" % k)

In [69]:
def print_groups(communities):
    html = "<table>"

    html += """
    <tr>
      <td></td>
      <th>~ philosophy</td>
      <th>~ knowledge</td>
      <th>~ wisdom</td>
      <th>~ love</td>
    </tr>
    """
    
    for c, philosophy, knowledge, wisdom, love in communities:
        html += "<tr><td style=\"width: 100px; text-align: center; \"><h3>group %s</h3></td>" % (c)
        html += "<td><ul>"
        html += "\n".join([ "<li>%s</li>" % word for word, score in philosophy ])
        html += "</ul></td>"
        html += "<td><ul>"
        html += "\n".join([ "<li>%s</li>" % word for word, score in knowledge ])
        html += "</ul></td>"
        html += "<td><ul>"
        html += "\n".join([ "<li>%s</li>" % word for word, score in wisdom ])
        html += "</ul></td>"
        html += "<td><ul>"
        html += "\n".join([ "<li>%s</li>" % word for word, score in love ])
        html += "</ul></td>"
        html += "</tr>"
    
    html += "</table>"

    display(HTML(html))

def q(query, community_id):
  model = models.word2vec.Word2Vec.load("models/word2vec/community.%s.txt" % community_id)
  
  try:
    result = model.most_similar(query)
  except:
    result = []
  
  return result

print_groups(map(lambda k: (k, q("philosophy", k), q("knowledge", k), q("wisdom", k), q("love", k)) , sorted([ int(k) for k in communities.keys()])))